# 产品内码与EPC码转换规则
EPC码标头二进制值设定为00010000

In [2]:
header = '00010000'

## 1、产品内码转EPC码
产品内码分为纯数字型（如:* 49531001649100 * ）和字母数字混合型( 如:* 24310100100148A * )。 
通过1位bit过滤值来区分。0表示数字型，1表示字母数字混合型。

In [3]:
import re

# 预定失败消息
fail = {'valid': False, }

# 异常
class HandleError(RuntimeError):
    def __init__(self, arg):
        self.args = arg

inner_num_code = '49531001649100'
inner_mix_code = '24310100100148A'

def filter(inner_code):
    filter = '0'
    if (re.match('^[\d]+$', inner_code)):
        filter = '0'
    elif (re.match('^(?!\d+$)[\da-zA-Z]+$',inner_code)):
        filter = '1'
    else:
        raise HandleError('产品内码不符合规范')    
    return filter


print('内码为{0}时，过滤值为{1}'.format(inner_num_code,filter(inner_num_code)))
print('内码为{0}时，过滤值为{1}'.format(inner_mix_code,filter(inner_mix_code)))      

内码为49531001649100时，过滤值为0
内码为24310100100148A时，过滤值为1


In [4]:
def padInnerCode(inner_code):
    if (filter(inner_code) == '0'):        
        return '1' + inner_code
    else:
        return inner_code
    
print('内码为{0}时，补位结果为{1}'.format(inner_num_code,padInnerCode(inner_num_code)))
print('内码为{0}时，补位结果为{1}'.format(inner_mix_code,padInnerCode(inner_mix_code)))

内码为49531001649100时，补位结果为149531001649100
内码为24310100100148A时，补位结果为24310100100148A


如果过滤值为0，则在内码高位补1

In [5]:
def padInnerCode(inner_code):
    if (filter(inner_code) == '0'):        
        return '1' + inner_code
    else:
        return inner_code
    
print('内码为{0}时，补位结果为{1}'.format(inner_num_code,padInnerCode(inner_num_code)))
print('内码为{0}时，补位结果为{1}'.format(inner_mix_code,padInnerCode(inner_mix_code)))

内码为49531001649100时，补位结果为149531001649100
内码为24310100100148A时，补位结果为24310100100148A


如果过滤值为0，直接将十进制字符串转为二进制字符串。如果过滤值为1，将字符串按位编码。
编码规则:
'0'\~'9'转为十进制的0\~9，'A'\~'Z'转为十进制的10\~35,'a'\~'z'转为十进制的36\~61,所以需要每个字符串需要6位即最大值2<sup>6</sup> = 64来表示所有的字符。

In [117]:
def innerCode2Bin(inner_code):
    if (filter(inner_code) == '0'): 
        print(bin(int(padInnerCode(inner_code)))[2:])
        return bin(int(padInnerCode(inner_code)))[2:]
    else:
        bin_str = ''
        for c in padInnerCode(inner_code):
            if (re.match('^[0-9]$',c)):
                bin_str += bin(int(c))[2:].zfill(6)
            elif (re.match('^[A-Z]$',c)):  
                bin_str += bin(ord(c) - 55)[2:].zfill(6)
            else:
                bin_str += bin(ord(c) - 61)[2:].zfill(6)
        return bin_str
    
print('内码为{0}时，二进制字符串为{1}'.format(inner_num_code,innerCode2Bin(inner_num_code)))
print('内码为{0}时，二进制字符串为{1}'.format(inner_mix_code,innerCode2Bin(inner_mix_code)))    

100001111111111101100110001111000111011111001100
内码为49531001649100时，二进制字符串为100001111111111101100110001111000111011111001100
内码为24310100100148A时，二进制字符串为000010000100000011000001000000000001000000000000000001000000000000000001000100001000001010


十进制序列号转为固定长度为38位的二进制字符串，

In [17]:
def serial2Bin(serial):
    return bin(int(serial))[2:].zfill(38)
serial = 123
print('序号为{0}的二进制字符串为{1}'.format(serial,serial2Bin(serial)))

序号为123的二进制字符串为00000000000000000000000000000001111011


计算二进制字符串总容量，总容量为 **标头** + **过滤** + **内码转换** + **序列号**

In [18]:
def capacity(inner_code):
    return len(header + filter(inner_code) + innerCode2Bin(inner_code) + serial2Bin('0'))

print('内码为{0}时，二进制字符串总容量为{1}'.format(inner_num_code,capacity(inner_num_code)))
print('内码为{0}时，二进制字符串总容量为{1}'.format(inner_mix_code,capacity(inner_mix_code)))

内码为49531001649100时，二进制字符串总容量为95
内码为24310100100148A时，二进制字符串总容量为137


根据EPC容量，调整 **内码转换** 的容量

内码字符串转成特定EPC容量的二进制字符串

In [48]:
def innerCodeSerial2Bin(inner_code, serial, epc_capacity):
    length = capacity(inner_code)
    if (epc_capacity < length):
        raise HandleError('超出EPC容量{0}bit'.format(epc_capacity))
    else:
        return header + filter(inner_code) + ''.zfill(epc_capacity - length) + innerCode2Bin(inner_code) + serial2Bin(serial)

epc_capacity = 96
print('内码为{0} EPC容量为{1}时，二进制字符串为{2}'.format(inner_num_code,epc_capacity,innerCodeSerial2Bin(inner_num_code,serial,epc_capacity)))
epc_capacity = 128
print('内码为{0} EPC容量为{1}时，二进制字符串为{2}'.format(inner_num_code,epc_capacity,innerCodeSerial2Bin(inner_num_code,serial,epc_capacity)))
epc_capacity = 256
print('内码为{0} EPC容量为{1}时，二进制字符串为{2}'.format(inner_num_code,epc_capacity,innerCodeSerial2Bin(inner_num_code,serial,epc_capacity)))
epc_capacity = 96
try:
    print('内码为{0} EPC容量为{1}时，二进制字符串为{2}'.format(inner_mix_code,epc_capacity,innerCodeSerial2Bin(inner_mix_code,serial,epc_capacity)))
except HandleError as err:   
    print("错误：" + "".join(list(map(str, err.args))))
epc_capacity = 128
try:
    print('内码为{0} EPC容量为{1}时，二进制字符串为{2}'.format(inner_mix_code,epc_capacity,innerCodeSerial2Bin(inner_mix_code,serial,epc_capacity)))
except HandleError as err:
    print("错误：" + "".join(list(map(str, err.args))))
epc_capacity = 256
print('内码为{0} EPC容量为{1}时，二进制字符串为{2}'.format(inner_mix_code,epc_capacity,innerCodeSerial2Bin(inner_mix_code,serial,epc_capacity)))

内码为49531001649100 EPC容量为96时，二进制字符串为000100000010000111111111110110011000111100011101111100110000000000000000000000000000000001111011
内码为49531001649100 EPC容量为128时，二进制字符串为00010000000000000000000000000000000000000010000111111111110110011000111100011101111100110000000000000000000000000000000001111011
内码为49531001649100 EPC容量为256时，二进制字符串为0001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000111111111110110011000111100011101111100110000000000000000000000000000000001111011
错误：超出EPC容量96bit
错误：超出EPC容量128bit
内码为24310100100148A EPC容量为256时，二进制字符串为0001000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000010000001100000100000000000100000000000000000100000000000000000100010000100000101000000000000000000000000000000001111011


In [111]:
# 二进制字符串转16进制字符串
def bin2Hex(bin_str):
    return hex(int(bin_str, 2))

print(bin2Hex('000100000010000111111111110110011000111100011101111100110000000000000000000000000000000001111011')[2:])
 
# 16进制字符串转二进制字符串
def hex2Bin(hex_str):
    bin_str = bin(int(hex_str,16))[2:]
    padding = (4-len(bin_str)%4)%4
    return '0'*padding + bin_str

print(hex2Bin('1021ffd98f1df3000000007b'))
    

1021ffd98f1df3000000007b
000100000010000111111111110110011000111100011101111100110000000000000000000000000000000001111011


EPC码转产品编码

In [138]:
def decodeInnerCode(bin_str):
    filter = bin_str[8:9]
    code_str = bin_str[10:len(bin_str)-38]
    serial = int(bin_str[len(bin_str)-38:],2)
    if (filter == '0'):
        print('1%38d'  )
        return str(int(code_str,2))[1:] + str(serial)
    else:
        return ''
    
def decodeBarCode(bin_str):
    print(1)

def decode(hex_str):
    bin_str = hex2Bin(hex_str)
    header = bin_str[:8]
    if (hex(int(header,2))[2:] == '10'):
        decodeInnerCode(bin_str)
    else:
        decodeBarCode(bin_str)
                

decode('1021ffd98f1df3000000007b')

49531001649100123
